In [1]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import itertools

import pyodbc

from dateutil.relativedelta import relativedelta
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from AMIEndEvents import AMIEndEvents

In [2]:
def daterange(start_date, end_date):
    r"""
    Just as the built-in range generator, but for dates.
    
    For consistency with the built-in range() function this iteration stops before reaching the end_date. 
    So for inclusive iteration use the next day, as you would with range().
    """
    for i in range((pd.to_datetime(end_date)-pd.to_datetime(start_date)).days):
        yield pd.to_datetime(start_date) + datetime.timedelta(days=i)


In [3]:
run_prem_nbs = False

save_dir_base_SNs      = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\SNs_with_end_events'
save_dir_base_prem_nbs = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\prem_nbs_with_end_events'

# Running one year at a time
date_0 = '2019-01-01'
date_1 = pd.to_datetime(date_0)+relativedelta(years=1)
# date_1 = '2022-07-01'

opco = 'oh'

#-------------------------
if not run_prem_nbs:
    save_dir_base = save_dir_base_SNs
else:
    save_dir_base = save_dir_base_prem_nbs

save_dir = os.path.join(save_dir_base, 'OPCO', opco)
save_name = f'{pd.to_datetime(date_0).strftime("%Y")}.pkl'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [4]:
col_of_interest = 'serialnumber'
return_col = 'serialnumbers'
if run_prem_nbs:
    col_of_interest = 'aep_premise_nb'
    return_col = 'prem_nbs'
#-------------------------
final_df = pd.DataFrame()
entries=[]
# NOTE: In SQL, BETWEEN is inclusive.
#       However, one must be careful, because without using strftime, the day would be defined including
#       the second (e.g., single_date=2018-01-01 00:00:00).  In which case, the returned end events would
#       only be those which occurred at that exact second.
#       Using strftime remedies this, as it makes e.g., single_date=2018-01-01
for single_date in daterange(date_0, date_1):
    single_date=single_date.strftime("%Y-%m-%d")
    print(single_date)
    sns_with_end_events = AMIEndEvents.get_end_event_distinct_fields(
        date_0=single_date, 
        date_1=single_date, 
        fields=[col_of_interest], 
        are_datetime=False, 
        addtnl_build_sql_function_kwargs = dict(opco='oh')
    )
    #-----
    # If no events are found, an empty pd.DataFrame is returned, which must be handled differently
    if sns_with_end_events.shape[0]==0:
        print(f'No events for date: {single_date}')
        entry_i = pd.Series(data=[[]], index=[single_date])
        entry_i.index.name = 'date'
    else:
        sns_with_end_events['date']=single_date
        entry_i = sns_with_end_events.groupby('date').apply(lambda x: natsorted(x[col_of_interest].tolist()))
    #-----
    entry_i.name=return_col
    #-----
    entries.append(entry_i)
#-------------------------
entries_df = pd.concat(entries).to_frame()
entries_df.to_pickle(os.path.join(save_dir, save_name))

2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07
2017-01-08
2017-01-09
2017-01-10
2017-01-11
2017-01-12
2017-01-13
2017-01-14
2017-01-15
2017-01-16
2017-01-17
2017-01-18
2017-01-19
2017-01-20
2017-01-21
2017-01-22
2017-01-23
2017-01-24
2017-01-25
2017-01-26
2017-01-27
2017-01-28
2017-01-29
2017-01-30
2017-01-31
2017-02-01
2017-02-02
2017-02-03
2017-02-04
2017-02-05
2017-02-06
2017-02-07
2017-02-08
2017-02-09
2017-02-10
2017-02-11
2017-02-12
2017-02-13
2017-02-14
2017-02-15
2017-02-16
2017-02-17
2017-02-18
2017-02-19
2017-02-20
2017-02-21
2017-02-22
2017-02-23
2017-02-24
2017-02-25
2017-02-26
2017-02-27
2017-02-28
2017-03-01
2017-03-02
2017-03-03
2017-03-04
2017-03-05
2017-03-06
2017-03-07
2017-03-08
2017-03-09
2017-03-10
2017-03-11
2017-03-12
2017-03-13
2017-03-14
2017-03-15
2017-03-16
2017-03-17
2017-03-18
2017-03-19
2017-03-20
2017-03-21
2017-03-22
2017-03-23
2017-03-24
2017-03-25
2017-03-26
2017-03-27
2017-03-28
2017-03-29
2017-03-30
2017-03-31
2017-04-01

In [5]:
print(os.path.join(save_dir, save_name))

C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\prem_nbs_with_end_events\OPCO\oh\2017.pkl
